# High tip classification - Random forest

## Dask + RAPIDS

 <img src="https://images.exxactcorp.com/CMS/landing-page/resource-center/supported-software/deep-learning/rapids/Rapids-Logo-lg.png" width="400" />

**Hardware**: 20 nodes - g4dn.xlarge (4 CPU, 16GB RAM; 1 GPU, 16GB GPU RAM)

In [1]:
from ml_utils import MLUtils

ml_utils = MLUtils(
    ml_task='high_tip',
    tool='rapids',
    model='random_forest',
)

# Load data and feature engineering

In [2]:
from dask.distributed import Client, wait
from dask import persist
from dask_saturn import SaturnCluster

cluster = SaturnCluster(n_workers=20, scheduler_size='xlarge', worker_size='g4dnxlarge')
client = Client(cluster)
cluster

[2020-08-08 19:24:12] INFO - dask-saturn | Cluster is ready


In [3]:
from IPython.display import display, HTML

gpu_links = f'''
<b>GPU Dashboard links</b>
<ul>
<li><a href="{client.dashboard_link}/individual-gpu-memory" target="_blank">GPU memory</a></li>
<li><a href="{client.dashboard_link}/individual-gpu-utilization" target="_blank">GPU utilization</a></li>
</ul>
'''
display(HTML(gpu_links))

In [4]:
import numpy as np
import pandas as pd
import dask_cudf
import dask.dataframe as dd

With `dask_cudf` version 0.15, will be able to do `dask_cudf.read_parquet(...)`

In [5]:
%%time
tip_train = dask_cudf.from_dask_dataframe(
    dd.read_parquet(f'{ml_utils.taxi_path}/data/ml/tip_train', engine='pyarrow'))
len(tip_train)

CPU times: user 1.78 s, sys: 477 ms, total: 2.26 s
Wall time: 21.4 s


219889897

In [6]:
tip_train.head()

,id,pickup_datetime,dropoff_datetime,pickup_taxizone_id,dropoff_taxizone_id,pickup_weekday,pickup_weekofyear,pickup_hour,pickup_minute,pickup_week_hour,passenger_count,tip_fraction
0,a2acf0e861b84476b1ba8b91f08448f4,2016-07-10 06:56:05,2016-07-10 06:59:53,263.0,236.0,6,27,6,56,150,1.0,0.600000
1,bac9b63e96f14ca9a4e9c236d709f5f2,2016-07-10 10:50:18,2016-07-10 10:55:21,142.0,163.0,6,27,10,50,154,5.0,0.226667
2,6ec52187eb4a4ab6ae3ce632030c0322,2016-07-10 10:50:18,2016-07-10 11:08:38,74.0,66.0,6,27,10,50,154,1.0,0.000000
3,21102720ac044762977f91b30f8857a1,2016-07-10 10:50:19,2016-07-10 10:55:38,161.0,142.0,6,27,10,50,154,1.0,0.166667
4,3d002b5aa2404553adc0ad79c776bf32,2016-07-10 10:50:19,2016-07-10 11:08:54,226.0,261.0,6,27,10,50,154,1.0,0.206957


<br>

We can use the full data with Dask + RAPIDS, so no need to sample here

# Random forest classifier

We'll train a classifier than can predict "high-tip" rides - those where the tip percent is >25%. Convert `X` to 32 bit precision for GPU processing

In [7]:
features = ml_utils.tip_vars.features
y_col = ml_utils.tip_vars.y_col
y_clf = ml_utils.tip_vars.y_clf
high_tip = ml_utils.tip_vars.high_tip

tip_train[y_clf] = (tip_train[y_col] > high_tip).astype('int32')
X = tip_train[features].astype('float32')
y = tip_train[y_clf]

In [8]:
y.unique().compute()

0    0
1    1
Name: high_tip, dtype: int32

Calling `persist()` performs all the data loading and pre-processing and holds the data in RAM for us to do the model training

In [9]:
%%time
X, y = persist(X, y)
_ = wait([X, y])

CPU times: user 886 ms, sys: 28.6 ms, total: 915 ms
Wall time: 18.1 s


In [10]:
from cuml.dask.ensemble import RandomForestClassifier
rfc = RandomForestClassifier(n_estimators=100, max_depth=10, seed=42)

In [11]:
%%time
with ml_utils.time_fit():
    _ = rfc.fit(X, y)

CPU times: user 438 ms, sys: 95.5 ms, total: 534 ms
Wall time: 1.44 s


## Predict on test set

And calculate metrics. Save predictions and metrics to S3.

In [12]:
tip_test = dask_cudf.from_dask_dataframe(
    dd.read_parquet(f'{ml_utils.taxi_path}/data/ml/tip_test', engine='pyarrow'))

tip_test[y_clf] = (tip_test[y_col] > high_tip).astype('int32')
X_test = tip_test[features].astype('float32')

preds = tip_test[['id', y_clf]].copy()
preds.columns = ['id', 'actual']
preds['predicted'] = rfc.predict_proba(X_test[features])[1]

In [13]:
%%time
preds = preds.persist()
_ = wait(preds)

CPU times: user 158 ms, sys: 4.03 ms, total: 162 ms
Wall time: 3.06 s


In [14]:
preds.head()

,id,actual,predicted
0,2e8f402e4dc44f2fae8b9328a237c4d2,0,0.0
1,5f067a4121244f42bf460867c23b39c9,0,0.0
2,60e8442d3d434df4959261905a279f55,0,0.0
3,2d1537ce2ed347778e078eaee7eacd44,0,0.0
4,13bb8a9ecbd04b559b7b9e40904026b0,0,0.0


In [15]:
%%time
preds_dd = preds.to_dask_dataframe()
ml_utils.write_predictions(preds_dd)

Writing predictions to 's3://saturn-titan/nyc-taxi/ml_results/predictions/high_tip__rapids__random_forest'
Done writing predictions
CPU times: user 74 ms, sys: 0 ns, total: 74 ms
Wall time: 4.97 s


<br>

Convert to single-GPU DataFrame using `compute()` because the Dask+RAPIDS implementation doesnt yet have `roc_auc_score`

In [16]:
from cuml.metrics import roc_auc_score

preds_cudf = preds.compute()
auc = roc_auc_score(preds_cudf.actual, preds_cudf.predicted)
ml_utils.write_metric_df('auc', auc)

,ml_task,tool,model,metric,value,fit_seconds
0,high_tip,rapids,random_forest,auc,0.5049967,1.436731
